# Categorize by merchant name

To run this notebook it is necessary to have the following packages installed:
- pandas
- beautifulsoup4
- requests

In [ ]:
import bs4
import pandas as pd
import requests
import time
import json

### Load needed functions

In [ ]:
def search_google(term: str):
    url = f'https://google.com/search?q={term}'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}

    request_result = requests.get(url, headers=headers)
    soup = bs4.BeautifulSoup(request_result.text,
                             "html.parser")

    return [s.getText() for s in soup.find_all("div", class_='VwiC3b yXK7lf MUxGbd yDYNvb lyLwlc lEBKkf')]

def get_query_category_message(commerce: str, commerce_description: str, cat_spect: str) -> str:
    return f'''Tengo la siguiente lista de categorías: {cat_spect}

    Tengo un comercio llamado "{commerce}" con las siguiente descripción: "{commerce_description}".

    ¿Cual sería su categoría?. Si no puedes categorizar, asigna el valor SIN CLASIFICAR.
    Responde únicamente con la categoría, sin correcciones ortográficas y que no acabe en punto.'''

def call_chat_completion(message: str) -> str:
    url = 'https://api.openai.com/v1/chat/completions'
    headers = {'Content-Type': 'application/json',
               'Authorization': f'Bearer {api_key}'}
    data = {
        "model": "gpt-3.5-turbo",
        "messages": [{"role": "user", "content": f"{message}"}],
        "temperature": 0,
        "max_tokens": 100
    }

    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        answer = response.json()['choices'][0]['message']['content']
        return answer
    else:
        raise Exception(f'Error: {response.text}')


def call_text_completion(message: str) -> str:
    url = 'https://api.openai.com/v1/completions'

    headers = {'Content-Type': 'application/json',
               'Authorization': f'Bearer {api_key}'}

    data = {
        "model": "text-davinci-003",
        "prompt": f"{message}",
        "temperature": 0,
        "max_tokens": 100
    }

    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        answer = response.json()['choices'][0]['text']
        return answer
    else:
        raise Exception(f'Error: {response.text}')

def select_category(_commerce: str) -> dict:
    commerce_description = search_google(_commerce)
    query_category_message = get_query_category_message(_commerce, commerce_description[0], cat_spect)
    category = call_chat_completion(query_category_message).strip()
    return {'commerce': commerce, 'category': category}

In [ ]:
### Load parameters

In [ ]:
merchant_names = ['LA CAVERNA DE VOLTIR', 'DOMINOS PIZZA CORDOBA']
cat_spect = json.dumps(["ALQUILER DE VEHÍCULOS",
                        "AUTOPISTAS GASOLINERAS Y PARKINGS",
                        "JUEGOS Y APUESTAS",
                        "MODA",
                        "DEPORTES Y JUGUETES",
                        "MENAJE DEL HOGAR Y ELECTRÓNICA",
                        "BRICOLAJE Y JARDINERÍA",
                        "GRANDES ALMACENES",
                        "MUEBLES Y DECORACIÓN",
                        "OCIO",
                        "RESTAURACIÓN",
                        "SUPERMERCADOS Y ALIMENTACIÓN",
                        "TRANSPORTE (NO AÉREO)",
                        "TELECOMUNICACIONES TELEVISIÓN E INTERNET",
                        "VEHíCULOS Y REPARACIONES",
                        "VIAJES HOTELES Y LÍNEAS AÉREAS",
                        "OPERACIONES CAJERO",
                        "COMPRA ONLINE",
                        "PRODUCTOS Y SERVICIOS DIGITALES",
                        "SALUD Y BELLEZA",
                        "EDUCACIÓN",
                        "SEGUROS",
                        "TASAS E IMPUESTOS",
                        "JOYERÍA",
                        "AGUA",
                        "GAS Y ELECTRICIDAD",
                        "COMUNIDADES DE PROPIETARIOS",
                        "ALQUILER",
                        "SEGURIDAD SOCIAL",
                        "CUOTAS Y ENTIDADES FINANCIERAS",
                        "SOCIEDADES BENÉFICAS",
                        "TRANSFERENCIAS DE SALIDA",
                        "COMISIONES",
                        "PRODUCTOS FINANCIEROS",
                        "CHEQUES",
                        "TRASPASOS",
                        "OPERACIONES EN OFICINA",
                        "SINDICATOS TRABAJADORES",
                        "COLEGIOS PROFESIONALES",
                        "REVISTAS LIBROS Y PRENSA",
                        "SISTEMAS DE SEGURIDAD",
                        "GESTIÓN INMOBILIARIA",
                        "PARTIDOS POLÍTICOS",
                        "ASESORES GESTORÍAS Y ABOGADOS",
                        "GASTO PAGO INMEDIATO",
                        "SERVICIOS VETERINARIOS Y PRODUCTOS PARA MASCOTAS",
                        "SEGUNDA MANO Y ANTIGÜEDADES",
                        "SERVICIOS AGRÍCOLAS",
                        "TINTORERÍAS, LAVANDERÍAS Y SERVICIOS DE ROPA",
                        "TRANSPORTE DE MERCANCÍAS Y SERVICIOS POSTALES",
                        "AMORTIZACIONES EVO BANCO",
                        "OPERACIONES CAJERO SIN TARJETA",
                        "OTROS GASTOS",
                        "SIN CLASIFICAR"], ensure_ascii=False)
api_key = 'insert your api-key'

In [ ]:
### Find categories

In [ ]:
categories = []
for i, commerce in enumerate(merchant_names, start=1):
    category = select_category(commerce)
    categories.append(category)
    if i % 3 == 0:
        time.sleep(60)

df = pd.DataFrame(data=categories)
df.head()